In [1]:
import os
import sys
sys.path.append(os.path.abspath('../..'))
from nowoe.data.api.smartwatch.utilities.preparations import *
import pandas
import plotly_express as px
import numpy
import json
import pickle
import gzip
import matplotlib.pyplot as plt
from copy import deepcopy
from functools import reduce
from tqdm import tqdm
from datetime import datetime, timezone
from typing import Dict, List, Tuple, Union, Any, Iterator
import torch.utils.data.dataloader
from nowoe.data.api.smartwatch.utilities.timestamp import get_utc_date_from_utc_timestamp
from nowoe.data.api.smartwatch.data_manager.module import SmartwatchDataManager
from nowoe.deep_learning.data.dataset.smartwatch_study.single_slice import get_dataloaders, SmartwatchStudySingleSliceDataset, single_slice_collate_fn
from nowoe.deep_learning.data.preprocessing.single_slice.normalization import MinMaxSingleSliceNormalization

In [2]:
normalizer = MinMaxSingleSliceNormalization(feature_names_per_data_source=dict(daily=['heart_rate_tsvalue']))

dataloaders = get_dataloaders(
    batch_size=50,
    root_dir='../../resources/warrior_wellness/Analysis/local_repo/',
    subject_splits={
        "train": [f'SWS_{i:02d}' for i in range(0,10)],
        "test": [f'SWS_{i:02d}' for i in range(10,15)]},
    dataset_config={
        'slice_lengths': [3600],
        'slice_time_step': (5 * 60),
        'label_milestone_per_window': 1.0,
        'metadata_cache_filepath': './dataset_cache/dataset-cache-2.pkl.gz',
        'no_cache': False,
        'parallel_threads': 10
    },
    sampler_configs=dict(
       train=dict(
           negative_sample_count=1000,
            positive_sample_count=500,
            target_variable='overall_quantized_stress_value',
           split_name="train"
       ),
       test=dict(
               negative_sample_count=200,
            positive_sample_count=100,
            target_variable='overall_quantized_stress_value',
           split_name="test"
       )
        )
)

2022-02-25 07:54:11,858 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - initializing data manager...
2022-02-25 07:56:48,013 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing the dataset...
2022-02-25 07:56:48,411 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.dataset - INFO - Loaded metadata from cache file: /home/shayan/phoenix/nowoe_framework/nowoe_framework/notebooks/smartwatch/dataset_cache/dataset-cache-2.pkl.gz
2022-02-25 07:56:48,411 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.interface - INFO - preparing samplers...
2022-02-25 07:56:48,551 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.sampler - WARNING - no samples for label 2.057142857142857 was found in the split: test
2022-02-25 07:56:48,551 - nowoe.deep_learning.data.dataset.smartwatch_study.single_slice.sampler - WARNING - no samples for label 2.314285714285714 was found in the split: t

In [3]:
normalizer.learn(dataloaders['train'])

2022-02-25 07:56:48,557 - nowoe.deep_learning.data.preprocessing.single_slice.normalization.base - INFO - [learning normalizers] sampling from the given dataloader...
100%|█████████████████████████████████████████| 160/160 [01:04<00:00,  2.47it/s]
2022-02-25 07:57:53,289 - nowoe.deep_learning.data.preprocessing.single_slice.normalization.base - INFO - [learning distributions] fitting models (of class <class 'sklearn.preprocessing._data.MinMaxScaler'>) to the distributions...
2022-02-25 07:57:53,297 - nowoe.deep_learning.data.preprocessing.single_slice.normalization.base - INFO - [learning distributions] completed.


In [4]:
my_iter = iter(dataloaders['test'])

In [5]:
batch = next(my_iter)

In [6]:
batch['slice'][1]['daily'].heart_rate_tsvalue

34591    83.0
34592    79.0
34593    79.0
34594    79.0
34595    79.0
         ... 
34826    66.0
34827    66.0
34828    67.0
34829    67.0
34830    67.0
Name: heart_rate_tsvalue, Length: 240, dtype: float64

In [7]:
batch_normalized = normalizer(batch)

In [8]:
batch_normalized['slice'][1]['daily'].heart_rate_tsvalue

34591    0.272109
34592    0.244898
34593    0.244898
34594    0.244898
34595    0.244898
           ...   
34826    0.156463
34827    0.156463
34828    0.163265
34829    0.163265
34830    0.163265
Name: heart_rate_tsvalue, Length: 240, dtype: float64